In [1]:
import pandas as pd

# Function to downcast numeric columns to save memory
def downcast(df):
    for col in df.select_dtypes(include=['float64']).columns:
        df[col] = pd.to_numeric(df[col], downcast='float')
    for col in df.select_dtypes(include=['int64']).columns:
        df[col] = pd.to_numeric(df[col], downcast='integer')
    return df

# Load the CSV file using pandas
columns_to_use = [
    '70000', 'D', 'N', 'F', 'MK15 9HP', 'WILLEN', 'MILTON KEYNES', 
    'MILTON KEYNES.1', 'MILTON KEYNES.2', '1995-07-07 00:00', 'A'
]

df = pd.read_csv('C:/Users/Guest01/Documents/Manpreet_thesis/Datasets/UK_property_price/UKarchive/202304.csv', usecols=columns_to_use)

# Rename the columns for ease of use
df = df.rename(columns={
    '70000': 'price', 
    'D': 'Property_Type', 
    'N': 'Old/New', 
    'F': 'Duration', 
    'MK15 9HP': 'Postcode', 
    'WILLEN': 'Locality', 
    'MILTON KEYNES': 'Town/City', 
    'MILTON KEYNES.1': 'District', 
    'MILTON KEYNES.2': 'County', 
    '1995-07-07 00:00': 'Date_of_Transfer', 
    'A': 'PPDCategory_Type'
})

# Apply downcasting to reduce memory usage immediately
df = downcast(df)

# Stratified sampling to reduce dataset size (75% sampling by 'Property_Type' and 'Town/City')
df_sampled = df.groupby(['Property_Type', 'Town/City'], group_keys=False).apply(lambda x: x.sample(frac=0.75, random_state=42)).reset_index(drop=True)

# Filter the dataset to include only standard residential sales (PPDCategory_Type 'A')
df_sampled = df_sampled[df_sampled['PPDCategory_Type'] == 'A']


C:\Users\Guest01\AppData\Local\Temp\ipykernel_17692\2973047249.py:38: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_sampled = df.groupby(['Property_Type', 'Town/City'], group_keys=False).apply(lambda x: x.sample(frac=0.75, random_state=42)).reset_index(drop=True)


In [2]:
# Handling missing values for categorical columns efficiently
categorical_columns = ['Property_Type', 'Old/New', 'Duration', 'Postcode', 'Locality', 'Town/City', 'District', 'County', 'PPDCategory_Type']
mode_values = {col: df_sampled[col].mode()[0] for col in categorical_columns}
df_sampled = df_sampled.fillna(mode_values)

# Fill missing 'price' values (numeric) with the median
df_sampled['price'] = df_sampled['price'].fillna(df_sampled['price'].median())

# Extract date features
df_sampled['Date_of_Transfer'] = pd.to_datetime(df_sampled['Date_of_Transfer'])
df_sampled['Year'] = df_sampled['Date_of_Transfer'].dt.year
df_sampled['Month'] = df_sampled['Date_of_Transfer'].dt.month
df_sampled['Day'] = df_sampled['Date_of_Transfer'].dt.day
df_sampled['DayOfWeek'] = df_sampled['Date_of_Transfer'].dt.dayofweek


In [3]:
# Function to apply frequency encoding for categorical variables
def optimized_frequency_encoding(df, categorical_columns):
    for col in categorical_columns:
        if df[col].dtype == 'object':
            freq_encoding = df[col].value_counts().to_dict()
            df[col + '_freq'] = df[col].map(freq_encoding)
            df = df.drop(columns=[col])  # Drop the original object column
    return df

# List of categorical columns
categorical_columns = ['Postcode', 'Property_Type', 'Old/New', 'Duration', 'Locality', 'Town/City', 'District', 'County', 'PPDCategory_Type']

# Apply frequency encoding to all categorical columns
df_sampled = optimized_frequency_encoding(df_sampled, categorical_columns)

# Unified downcast function for all numeric columns
def downcast_all_numeric(df):
    df[df.select_dtypes(include=['float64']).columns] = df.select_dtypes(include=['float64']).apply(pd.to_numeric, downcast='float')
    df[df.select_dtypes(include=['int64']).columns] = df.select_dtypes(include=['int64']).apply(pd.to_numeric, downcast='integer')
    return df

# Downcast all numeric columns (including frequency-encoded columns)
df_sampled = downcast_all_numeric(df_sampled)

In [4]:
# Define features (X) and target (y)
X = df_sampled.drop(columns=['price', 'Date_of_Transfer'])  # Drop the target 'price' and the raw date column
y = df_sampled['price']

# Check data types to ensure all columns are numeric
print(f"Data types in X:\n{X.dtypes}")
if X.select_dtypes(include=['object']).empty:
    print("All columns are numeric.")
else:
    print("Some columns are still non-numeric.")

# Print the shape of X and y
print(f"Shape of X: {X.shape}")
print(f"Shape of y: {y.shape}")


Data types in X:
Year                     int32
Month                    int32
Day                      int32
DayOfWeek                int32
Postcode_freq            int16
Property_Type_freq       int32
Old/New_freq             int32
Duration_freq            int32
Locality_freq            int32
Town/City_freq           int32
District_freq            int32
County_freq              int32
PPDCategory_Type_freq    int32
dtype: object
All columns are numeric.
Shape of X: (20274118, 13)
Shape of y: (20274118,)


In [5]:
from sklearn.model_selection import train_test_split

# Function to sample data
def sample_data(X, y, sample_size):
    if isinstance(sample_size, float):
        if 0 < sample_size < 1.0:
            return train_test_split(X, y, test_size=0.2, train_size=sample_size, random_state=42)
        elif sample_size == 1.0:
            return train_test_split(X, y, test_size=0.001, random_state=42)
        else:
            raise ValueError("sample_size as float must be in the range (0.0, 1.0) or equal to 1.0.")
    elif isinstance(sample_size, int):
        if sample_size > len(X):
            raise ValueError(f"sample_size {sample_size} exceeds the number of available samples {len(X)}.")
        sampled_X = X.sample(n=sample_size, random_state=42)
        sampled_y = y.loc[sampled_X.index]
        return train_test_split(sampled_X, sampled_y, test_size=0.2, random_state=42)
    else:
        raise ValueError("sample_size must be a float or an integer")

In [6]:
import numpy as np
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
import time
import psutil

# Function to calculate and return metrics for ElasticNet Regressor
def calculate_metrics(X_train, X_test, y_train, y_test):
    model = ElasticNet()

    # Define the parameter grid for RandomizedSearchCV using np.linspace
    # param_grid = {
    #     'alpha': np.linspace(0.1, 1, 20),  # 20 values between 0.1 and 1
    #     'l1_ratio': np.linspace(0.1, 0.9, 20),  # 20 values between 0.1 and 0.9
    #     'fit_intercept': [True, False],
    #     'max_iter': [1000, 5000],
    #     'tol': [1e-4, 1e-3]
    # }

    param_grid = {
    'alpha': np.linspace(0.001, 1, 5),  # Range between 0.001 and 1 to ensure positive values
    'l1_ratio': np.linspace(0.1, 0.9, 5),  # Range for balancing L1 and L2 regularization
    'fit_intercept': [True, False],
    'max_iter': [500, 1000],  # Lower max_iter to speed up training
    'tol': [1e-3, 1e-2]  # Slightly larger tolerance for faster convergence
}

    # Define RandomizedSearchCV with n_iter=20
    random_search = RandomizedSearchCV(model, param_distributions=param_grid, n_iter=10, cv=3, scoring='neg_mean_squared_error', n_jobs=-1)

    # Measure execution time and CPU usage
    start_time = time.time()
    start_cpu = psutil.cpu_percent(interval=None)

    # Fit the model
    random_search.fit(X_train, y_train)

    end_cpu = psutil.cpu_percent(interval=None)
    end_time = time.time()

    # Calculate time and CPU usage
    execution_time = end_time - start_time
    avg_cpu_usage = (start_cpu + end_cpu) / 2

    # Make predictions
    y_pred = random_search.best_estimator_.predict(X_test)

    # Calculate metrics
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mape = mean_absolute_percentage_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    # Calculate the range of the target variable for normalized RMSE
    target_range = y_train.max() - y_train.min()
    nrmse = rmse / target_range

    # Calculate memory usage in MB
    memory_usage_MB = X_train.memory_usage(deep=True).sum() / (1024 ** 2)
    normalized_time = execution_time / memory_usage_MB

    return {
        'RMSE': rmse,
        'MAPE': mape,
        'R2': r2,
        'nRMSE': nrmse,  # Normalized RMSE
        'Execution Time (Raw)': execution_time,  # Raw execution time
        'Normalized Time (s/MB)': normalized_time,  # Normalized execution time
        'Average CPU Usage': avg_cpu_usage
    }

import gc # Garbage Collector

# Define sample sizes
sample_sizes = [1.0, 0.5, 0.25, 0.125, 100, 1000, 10000, 100000]

# Initialize metrics storage
metrics_list = []
total_execution_time = 0
total_cpu_usage = 0
total_memory_usage_MB = 0

# Loop through each sample size
for size in sample_sizes:
    try:
        X_train_sample, X_test_sample, y_train_sample, y_test_sample = sample_data(X, y, size)
        metrics = calculate_metrics(X_train_sample, X_test_sample, y_train_sample, y_test_sample)
        metrics['Sample Size'] = size
        metrics_list.append(metrics)

        # Call garbage collection after each iteration to free up memory
        gc.collect()    

        # Accumulate total       
        total_execution_time += metrics['Execution Time (Raw)']
        total_cpu_usage += metrics['Average CPU Usage']
        total_memory_usage_MB += X_train_sample.memory_usage(deep=True).sum() / (1024 ** 2)

        print(f"Metrics for sample size {size}:")
        for key, value in metrics.items():
            print(f"{key}: {value}")
        print("-" * 50)

    except Exception as e:
        print(f"An error occurred for sample size {size}: {e}")

c:\Users\Guest01\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.433e+17, tolerance: 1.217e+16
  model = cd_fast.enet_coordinate_descent(


Metrics for sample size 1.0:
RMSE: 220074.31663096903
MAPE: 0.691200164823368
R2: 0.22223189321117942
nRMSE: 0.002445270211958214
Execution Time (Raw): 10662.248704433441
Normalized Time (s/MB): 9.51728969836524
Average CPU Usage: 58.050000000000004
Sample Size: 1.0
--------------------------------------------------


c:\Users\Guest01\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.980e+14, tolerance: 6.074e+14
  model = cd_fast.enet_coordinate_descent(


Metrics for sample size 0.5:
RMSE: 216572.6780369479
MAPE: 0.7610245697189799
R2: 0.23189294625547263
nRMSE: 0.0038673693197199504
Execution Time (Raw): 3185.3745222091675
Normalized Time (s/MB): 5.680943207050214
Average CPU Usage: 47.25
Sample Size: 0.5
--------------------------------------------------


c:\Users\Guest01\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.466e+16, tolerance: 3.002e+14
  model = cd_fast.enet_coordinate_descent(


Metrics for sample size 0.25:
RMSE: 216572.7677327143
MAPE: 0.7609518361961981
R2: 0.23189231001672128
nRMSE: 0.0040587101160311926
Execution Time (Raw): 2830.3967373371124
Normalized Time (s/MB): 10.095720043163444
Average CPU Usage: 56.65
Sample Size: 0.25
--------------------------------------------------
Metrics for sample size 0.125:
RMSE: 216572.75049496093
MAPE: 0.7617350937180744
R2: 0.23189243228925382
nRMSE: 0.0041741969002661574
Execution Time (Raw): 1165.0826725959778
Normalized Time (s/MB): 8.31145023252075
Average CPU Usage: 51.949999999999996
Sample Size: 0.125
--------------------------------------------------
Metrics for sample size 100:
RMSE: 140432.97966598877
MAPE: 0.7019592094908051
R2: -0.1871322877949777
nRMSE: 0.14605614109827225
Execution Time (Raw): 0.08215188980102539
Normalized Time (s/MB): 18.565193965517242
Average CPU Usage: 39.65
Sample Size: 100
--------------------------------------------------
Metrics for sample size 1000:
RMSE: 131130.75507982998
MAP

In [7]:
# Convert metrics to DataFrame
metrics_df = pd.DataFrame(metrics_list)

# Calculate total metrics
total_avg_cpu_usage = total_cpu_usage / len(sample_sizes)
normalized_total_time = total_execution_time / total_memory_usage_MB

# Convert total execution time to minutes and seconds
total_minutes = int(total_execution_time // 60)
total_seconds = total_execution_time % 60

# Display total metrics
print(f"Total Execution Time for Entire Process (Raw): {total_minutes} minutes and {total_seconds:.2f} seconds")
print(f"Total Normalized Execution Time for Entire Process: {normalized_total_time:.8f} seconds per MB")
print(f"Total Average CPU Usage for Entire Process: {total_avg_cpu_usage:.2f}%")

# Display the metrics DataFrame
metrics_df

Total Execution Time for Entire Process (Raw): 297 minutes and 34.82 seconds
Total Normalized Execution Time for Entire Process: 8.47619642 seconds per MB
Total Average CPU Usage for Entire Process: 44.06%


,RMSE,MAPE,R2,nRMSE,Execution Time (Raw),Normalized Time (s/MB),Average CPU Usage,Sample Size
0,220074.316631,0.691200,0.222232,0.002445,10662.248704,9.517290,58.05,1.000
1,216572.678037,0.761025,0.231893,0.003867,3185.374522,5.680943,47.25,0.500
2,216572.767733,0.760952,0.231892,0.004059,2830.396737,10.095720,56.65,0.250
3,216572.750495,0.761735,0.231892,0.004174,1165.082673,8.311450,51.95,0.125
4,140432.979666,0.701959,-0.187132,0.146056,0.082152,18.565194,39.65,100.000
5,131130.755080,0.611359,0.243382,0.061942,1.610527,36.395690,22.75,1000.000
6,173824.775828,0.728822,0.289306,0.012465,0.383545,0.866760,25.70,10000.000
7,181507.249281,0.716252,0.281927,0.008250,9.637948,2.178043,50.50,100000.000
